In [5]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c, e, m_p
from scipy.signal import fftconvolve

%matplotlib tk
import seaborn as sns
sns.set_context('notebook', font_scale=1.5,
                rc={'lines.markeredgewidth': 1})
sns.set_style('darkgrid', {
        'axes.linewidth': 2,
        'legend.fancybox': True})

In [6]:
from SPS import SPS

Here we can play with **tunes** and **n_segments** - really the combination of the two, plus the type of non-linearity, determine the resonances excited...

In [146]:
# =====================================
# Input parameters
Qx, Qy = 20.24, 20.18
n_segments = 3#9
n_turns = 2048

# =====================================
# Build machine
C = np.pi*50*44
# sps = SPS(machine_configuration='Q20-flattop', RF_at='end_of_transverse', optics_mode='non-smooth',
#           name=['s{:d}'.format(i) for i in xrange(n_segments+1)],
#           s=[i*C/n_segments for i in xrange(n_segments+1)],
#           accQ_x=[i*Qx/n_segments for i in xrange(n_segments+1)],
#           accQ_y=[i*Qy/n_segments for i in xrange(n_segments+1)],
#           beta_x=np.array([50, 100, 10, 50]), beta_y=np.array([50, 10, 100, 50]),
#           alpha_x=np.zeros(n_segments+1), alpha_y=np.zeros(n_segments+1)
#           )
sps = SPS(machine_configuration='Q20-flattop', RF_at='end_of_transverse',
          n_segments=n_segments, accQ_x=Qx, accQ_y=Qy)
sps.one_turn_map.pop(-1)
ax = sps.transverse_map.alpha_x[0]
ay = sps.transverse_map.alpha_y[0]
bx = sps.transverse_map.beta_x[0]
by = sps.transverse_map.beta_y[0]

# =====================================
# Add octupoles
from PyHEADTAIL.multipoles.multipoles import ThinSextupole, ThinOctupole

# sex = ThinSextupole(k2l=2)
# sps.install_after_each_transverse_segment(sex)
oct = ThinOctupole(k3l=1000)
sps.install_after_each_transverse_segment(oct)

Synchrotron init. From kwargs: n_segments = 3
Synchrotron init. From kwargs: accQ_y = 20.18
Synchrotron init. From kwargs: accQ_x = 20.24
Synchrotron init. From kwargs: RF_at = 'end_of_transverse'


In [147]:
macroparticlenumber = 500
bunch = sps.generate_6D_Gaussian_bunch(n_macroparticles=macroparticlenumber, intensity=1e11,
                                       epsn_x=3.5e-6, epsn_y=3.5e-6, sigma_z=0.23)
# bunch.update({
#         'x': xx.flatten(),
#         'xp': xxp.flatten(),
#         'y': yy.flatten(),
#         'yp': yyp.flatten()})

x = np.zeros((n_turns, macroparticlenumber))
xp = np.zeros((n_turns, macroparticlenumber))
y = np.zeros((n_turns, macroparticlenumber))
yp = np.zeros((n_turns, macroparticlenumber))

# ===============
# Do the tracking
for i in xrange(n_turns):
    for m in sps.one_turn_map:
        m.track(bunch)

    x[i,:] = bunch.x
    xp[i,:] = bunch.xp
    y[i,:] = bunch.y
    yp[i,:] = bunch.yp
    
# =======================
# Some frequency analysis
from PySussix import Sussix

tunesx1 = np.zeros(macroparticlenumber)
tunesy1 = np.zeros(macroparticlenumber)
tunesx2 = np.zeros(macroparticlenumber)
tunesy2 = np.zeros(macroparticlenumber)

ssx = Sussix()

ssx.sussix_inp(nt1=1, nt2=n_turns/2, idam=2, ir=0, tunex=0.2, tuney=0.2)
for i in xrange(macroparticlenumber):
    if i%20==0:
        sys.stdout.write("\r--> Processing particle {:d}".format(i))
    ssx.sussix(x[:,i], bx*xp[:,i], y[:,i], by*yp[:,i], x[:,i], xp[:,i])
    tunesx1[i] = ssx.ox[np.argmax(ssx.ax)]
    tunesy1[i] = ssx.oy[np.argmax(ssx.ay)]

ssx.sussix_inp(nt1=n_turns/2, nt2=n_turns, idam=2, ir=0, tunex=0.2, tuney=0.2)
for i in xrange(macroparticlenumber):
    if i%20==0:
        sys.stdout.write("\r--> Processing particle {:d}".format(i))
    ssx.sussix(x[:,i], bx*xp[:,i], y[:,i], by*yp[:,i], x[:,i], xp[:,i])
    tunesx2[i] = ssx.ox[np.argmax(ssx.ax)]
    tunesy2[i] = ssx.oy[np.argmax(ssx.ay)]
    
diff_x = np.log(tunesx2-tunesx1)
diff_y = np.log(tunesy2-tunesy1)


plt.close('all')
# ===============
# Plot output
fig = plt.figure(figsize=(20, 10))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132, sharex=ax1, sharey=ax1)
ax3 = fig.add_subplot(133)

ax1.plot(x, xp, '.')
ax2.plot(y, yp, '.')
ax1.set_xlim(-1e-2, 1e-2)
ax1.set_ylim(-1e-4, 1e-4)

ax3.scatter(tunesx1, tunesy1, c=diff_x, s=40, marker='o', cmap='viridis')
ax3.set_xlim(left=Qx%1-4e-2)
ax3.set_ylim(bottom=Qy%1-4e-2)

plt.show()

--> Processing particle 480

***

## Regular points

In [119]:
bunch = sps.generate_6D_Gaussian_bunch(n_macroparticles=1e5, intensity=1e11,
                                       epsn_x=4.5e-6, epsn_y=4.5e-6, sigma_z=0.23)
sJx = np.sqrt(bunch.sigma_x()**2 + (bx*bunch.sigma_xp())**2)/2.
sJy = np.sqrt(bunch.sigma_x()**2 + (bx*bunch.sigma_xp())**2)/2.
print sJx, sJy


J = np.linspace(0, 1, 12)**2 * 3.6e-8
phi1 = np.linspace(0, 2*np.pi, 12)
phi2 = np.linspace(0, 2*np.pi, 12)
phi3 = np.linspace(0, 2*np.pi, 12)

JJ, PP = np.meshgrid(J, phi1)
xx = np.sqrt(2.*JJ*bx)*np.cos(PP)
xxp = -np.sqrt(2.*JJ/bx)*(np.sin(PP) + ax*np.cos(PP))
yy = np.sqrt(2.*JJ*by)*np.sin(PP)
yyp = -np.sqrt(2.*JJ/by)*(np.cos(PP) + ay*np.sin(PP))


macroparticlenumber = np.product(JJ.shape)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,9))
ax1.plot(xx, yy, '-o')
ax2.plot(yyp, xxp, '-o')

plt.show()

0.000485036753365 0.000485036753365


## Build a 4-sphere grid

In [123]:
macroparticlenumber = 400

u4 = np.random.normal(size=(4, macroparticlenumber))
r  = np.sqrt(sum(u4**2))
u4 *= 1./r * 7e-5
xx = u4[0,:]
xxp = u4[1,:]
yy = u4[2,:]
yyp = u4[3,:]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,9))
ax1.plot(xx, yy, 'o')
ax2.plot(xxp, yyp, 'o')

plt.show()

In [100]:
bunch = sps.generate_6D_Gaussian_bunch(n_macroparticles=1e5, intensity=1e11,
                                       epsn_x=4.5e-6, epsn_y=4.5e-6, sigma_z=0.23)
sJx = np.sqrt(bunch.sigma_x()**2 + (bx*bunch.sigma_xp())**2)/2.
sJy = np.sqrt(bunch.sigma_x()**2 + (bx*bunch.sigma_xp())**2)/2.
print sJx, sJy


J = np.linspace(0, 1, 12)**2 * 3.6e-8
phi1 = np.linspace(0, 2*np.pi, 12)
phi2 = np.linspace(0, 2*np.pi, 12)
phi3 = np.linspace(0, 2*np.pi, 12)

JJ, PP = np.meshgrid(J, phi1)
xx = np.sqrt(2.*JJ*bx)*np.cos(PP)
xxp = -np.sqrt(2.*JJ/bx)*(np.sin(PP) + ax*np.cos(PP))
yy = np.sqrt(2.*JJ*by)*np.cos(PP)
yyp = -np.sqrt(2.*JJ/by)*(np.sin(PP) + ay*np.cos(PP))


macroparticlenumber = np.product(JJ.shape)

x = np.zeros((n_turns, macroparticlenumber))
xp = np.zeros((n_turns, macroparticlenumber))
y = np.zeros((n_turns, macroparticlenumber))
yp = np.zeros((n_turns, macroparticlenumber))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,9))
ax1.plot(xx, yy, '-o')
ax2.plot(yy, yyp, '-o')

plt.show()

0.000483725204433 0.000483725204433


***